# Applying normal Timewise PCA

In [2]:
import seaborn as sns
import os
import pandas as pd
from matplotlib.pyplot import title

sns.set_theme(style="ticks")

In [3]:
# Load segmented data
def load_segmented_data(data_folder="", event_name=None):
    """
    Load segmented CSV data.

    Parameters:
    - data_folder (str): Folder where the segmented .csv files are stored.
    - event_name (str or None): Name of the event to load. If None, loads all CSV files.

    Returns:
    - pd.DataFrame: Loaded dataframe with all data concatenated if loading all events.
    """
    if event_name is not None:
        # Load one specific event csv
        file_path = os.path.join(data_folder, f"segment_around_{event_name}.csv")
        if not os.path.isfile(file_path):
            raise FileNotFoundError(f"No data found for event '{event_name}' at {file_path}")
        df = pd.read_csv(file_path)
        print(f"Loaded data for event '{event_name}' from {file_path}.")
        return df
    else:
        # Load all csv files in the folder that match pattern 'segment_around_*.csv'
        all_files = [f for f in os.listdir(data_folder) if f.startswith("segment_around_") and f.endswith(".csv")]
        if not all_files:
            raise FileNotFoundError(f"No segmented CSV files found in folder {data_folder}")

        dfs = []
        for file in all_files:
            path = os.path.join(data_folder, file)
            df_temp = pd.read_csv(path)
            # Optionally add a column indicating source event, parsed from filename
            event_from_file = file.replace("segment_around_", "").replace(".csv", "")
            df_temp['EventName'] = event_from_file
            dfs.append(df_temp)
        df_all = pd.concat(dfs, ignore_index=True)
        print(f"Loaded and concatenated {len(all_files)} segmented event files from {data_folder}.")
        return df_all

In [4]:
# Data path
data_folder = "data/cleaned_data/data_segment/Safak"
# Load single event:
df_single_event = load_segmented_data(data_folder=data_folder, event_name="StagEventNew_nearest_neighbor_steeringRemoved")
df_single_event
# Load all events:
# df_all = load_segmented_data(data_folder="data/data_segment")

Loaded data for event 'StagEventNew_nearest_neighbor_steeringRemoved' from data/cleaned_data/data_segment/Safak/segment_around_StagEventNew_nearest_neighbor_steeringRemoved.csv.


,timestamp_REF,uid,dataset,city_section,ExperimentalCondition,UnixTimeStamp,TobiiTimeStamp,RightEyeIsBlinkingWorld,RightEyeIsBlinkingLocal,LeftEyeIsBlinkingWorld,...,TimeStamp,TimeStamp_in_sec,TimeDiff_in_sec,Blink,BlinkDuration,BlinkDurationsLong,BlinkAfter,SamplingRate,time_from_event,outlier
0,2020-09-26 13:46:22.260000+00:00,0037b2329de444c18d751b4e79901b39,EyeTracking,MountainRoad,BaseCondition,1.601128e+09,347.918274,False,False,False,...,1.601128e+09,139.759740,0.010971,False,0.106051,0.0,False,50.0,-5.00,False
1,2020-09-26 13:46:22.280000+00:00,0037b2329de444c18d751b4e79901b39,EyeTracking,MountainRoad,BaseCondition,1.601128e+09,347.939484,False,False,False,...,1.601128e+09,139.781682,0.021942,False,0.105959,0.0,False,50.0,-4.98,False
2,2020-09-26 13:46:22.300000+00:00,0037b2329de444c18d751b4e79901b39,EyeTracking,MountainRoad,BaseCondition,1.601128e+09,347.961914,False,False,False,...,1.601128e+09,139.803248,0.021566,False,0.105866,0.0,False,50.0,-4.96,False
3,2020-09-26 13:46:22.320000+00:00,0037b2329de444c18d751b4e79901b39,EyeTracking,MountainRoad,BaseCondition,1.601128e+09,347.983795,False,False,False,...,1.601128e+09,139.825190,0.016456,False,0.105774,0.0,False,50.0,-4.94,False
4,2020-09-26 13:46:22.340000+00:00,0037b2329de444c18d751b4e79901b39,EyeTracking,MountainRoad,BaseCondition,1.601128e+09,348.000427,False,False,False,...,1.601128e+09,139.841646,0.016705,False,0.105681,0.0,False,50.0,-4.92,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80155,2020-09-19 10:22:15.060000+00:00,ff4288f304e74bbf93aa6508c7df8145,EyeTracking,MountainRoad,FullLoopAR,1.600511e+09,717.885681,False,False,True,...,1.600511e+09,170.287970,0.009973,False,NaN,0.0,False,50.0,4.92,False
80156,2020-09-19 10:22:15.080000+00:00,ff4288f304e74bbf93aa6508c7df8145,EyeTracking,MountainRoad,FullLoopAR,1.600511e+09,717.907898,False,False,True,...,1.600511e+09,170.310909,0.022938,False,NaN,0.0,False,50.0,4.94,False
80157,2020-09-19 10:22:15.100000+00:00,ff4288f304e74bbf93aa6508c7df8145,EyeTracking,MountainRoad,FullLoopAR,1.600511e+09,717.919006,False,False,True,...,1.600511e+09,170.322876,0.010971,False,NaN,0.0,False,50.0,4.96,False
80158,2020-09-19 10:22:15.120000+00:00,ff4288f304e74bbf93aa6508c7df8145,EyeTracking,MountainRoad,FullLoopAR,1.600511e+09,717.952576,False,False,True,...,1.600511e+09,170.355788,0.022938,False,NaN,0.0,False,50.0,4.98,True


In [5]:
df_single_event['uid'].nunique()

160

In [6]:
df_single_event.columns

Index(['timestamp_REF', 'uid', 'dataset', 'city_section',
       'ExperimentalCondition', 'UnixTimeStamp', 'TobiiTimeStamp',
       'RightEyeIsBlinkingWorld', 'RightEyeIsBlinkingLocal',
       'LeftEyeIsBlinkingWorld', 'LeftEyeIsBlinkingLocal', 'HmdPosition.x',
       'HmdPosition.y', 'HmdPosition.z', 'NoseVector.x', 'NoseVector.y',
       'NoseVector.z', 'EyePosWorldCombined.x', 'EyePosWorldCombined.y',
       'EyePosWorldCombined.z', 'EyeDirWorldCombined.x',
       'EyeDirWorldCombined.y', 'EyeDirWorldCombined.z',
       'EyePosLocalCombined.x', 'EyePosLocalCombined.y',
       'EyePosLocalCombined.z', 'EyeDirLocalCombined.x',
       'EyeDirLocalCombined.y', 'EyeDirLocalCombined.z', 'ObjectName_1',
       'ObjectName_2', 'ObjectName_3', 'ObjectName_4', 'ObjectName_5',
       'HitObjectPosition.x_1', 'HitObjectPosition.x_2',
       'HitObjectPosition.x_3', 'HitObjectPosition.x_4',
       'HitObjectPosition.x_5', 'HitObjectPosition.y_1',
       'HitObjectPosition.y_2', 'HitObjectPositio

# Remap 'SuccessfulCompletionState'
This is important in case we want to use Success later for analysis etc.

In [7]:
# To see if all values are NaN, only 1, only 0, or mixed
result = df_single_event.groupby('uid')['SuccessfulCompletionState'].agg(
    lambda x: {
        'all_nan': x.isna().all(),
        'all_one': (x == 1).all() if not x.isna().all() else False,
        'all_zero': (x == 0).all() if not x.isna().all() else False,
        'mixed': not (x.isna().all() or (x == 1).all() or (x == 0).all())
    }
)
# To see the actual status info for each uid:
print(result)

uid
0037b2329de444c18d751b4e79901b39    {'all_nan': False, 'all_one': False, 'all_zero...
0117810eb9634c4f98f842021ee6a595    {'all_nan': False, 'all_one': False, 'all_zero...
0121f5b2f59d434f8beb17bf3e2a80b9    {'all_nan': False, 'all_one': False, 'all_zero...
0956f0cca5f546d79a0cf4fbae23d496    {'all_nan': False, 'all_one': False, 'all_zero...
09a23914cf354ea39444511406d16722    {'all_nan': False, 'all_one': False, 'all_zero...
                                                          ...                        
f9c6ff61370141c89ea9bbc536d796e1    {'all_nan': False, 'all_one': False, 'all_zero...
fa2e2604ec6a4820851f032e80f09ba1    {'all_nan': False, 'all_one': False, 'all_zero...
fa4ca90c5b80445b9af0b7ec4fbcc124    {'all_nan': False, 'all_one': False, 'all_zero...
fd19a21a4a8846ca82bd127e4e1933f5    {'all_nan': False, 'all_one': False, 'all_zero...
ff4288f304e74bbf93aa6508c7df8145    {'all_nan': False, 'all_one': False, 'all_zero...
Name: SuccessfulCompletionState, Length: 160, dtyp

In [8]:
# 1. Get the known value (0 or 1) for each uid
# First, identify the non-NaN value per 'uid' (assumes consistent non-NaN)
uid_value = df_single_event.groupby('uid')['SuccessfulCompletionState'].transform(lambda x: x.dropna().unique()[0])

# 2. Fill NaNs with the corresponding uid's value
# For entries where 'SuccessfulCompletionState' is NaN, replace with uid_value
df_single_event['SuccessfulCompletionState'] = df_single_event['SuccessfulCompletionState'].fillna(uid_value)
# To see if all values are NaN, only 1, only 0, or mixed
df_single_event['SuccessfulCompletionState']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
80155    1.0
80156    1.0
80157    1.0
80158    1.0
80159    1.0
Name: SuccessfulCompletionState, Length: 80160, dtype: float64

# 1. Apply PCA

In [9]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
import copy
import numpy as np
import pandas as pd
import dask.array as da
import scipy as sp
from scipy.stats import chi2
import gc
import seaborn as sns
from scipy.stats import chi2
from sklearn.covariance import MinCovDet
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.preprocessing import StandardScaler
from matplotlib.lines import Line2D
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)
plt.style.use('ggplot')

### Useful functions

In [10]:
# Simple mahalanobis distance
def mahalanobis(df, no_dask=True, md_name='',outlier_name=''):
    # Calculate mean
    x_minus_mu = df - np.mean(df)
    # Calculate covariance
    cov = np.cov(df.values.T)
    # Calculate inverse covariance
    if df.shape[1] < 2:
        # Reshape when only one column in used (e.g., 'SteeringInput')
        inv_covmat = sp.linalg.inv(cov.reshape((1,1)))
    else:
        inv_covmat = sp.linalg.inv(cov)
    if no_dask:
        # No dask work only for individual UID
        left_term = np.dot(x_minus_mu, inv_covmat)
        mahal = np.dot(left_term, x_minus_mu.T)
        # Calculate md
        md = np.sqrt(mahal.diagonal())
    else:
        # Dask needs to be used when calculating md across conditions
        x_minus_mu_dask = da.from_array(x_minus_mu.to_numpy(), chunks=(min(x_minus_mu.shape[0], 10000), df.shape[1]))
        mahal = np.sqrt(np.diagonal(np.dot(np.dot(x_minus_mu_dask,inv_covmat), x_minus_mu_dask.T)))
        # Calculate md
        md = mahal.compute()
    # Save md values to df column
    md_df = pd.DataFrame({md_name:md})
    # Set a cut-off threshold based on the Chi2 and the degrees of freedom
    # Here we choose to be conservative and only get very extreme outliers
    threshold = chi2.ppf((1-0.001), df=df.shape[1])  # degrees of freedom = number of variables
    # Assuming 'mahalanobis_distances' contains your computed distances
    # mean_md = np.mean(md)
    # std_md = np.std(md)
    # threshold_5std = mean_md + 3 * std_md
    # Set as outliers md > threshold
    md_df[outlier_name] = md_df[md_name] > threshold
    # if md_name == 'md_car':
    #     print(C)
    return md_df

In [11]:
### Robust Mahalonibis Distance
def robust_mahalanobis_method_dask(df, md_name='',outlier_name='', p_md_name='', cut=0.001):
    #Minimum covariance determinant
    rng = np.random.RandomState(0)
    real_cov = np.cov(df.values.T)

    # Calculate inverse covariance
    if df.shape[1] < 2:
        # Reshape when only one column in used (e.g., 'SteeringInput')
        # X = rng.multivariate_normal(mean=np.mean(df, axis=0), cov=real_cov.reshape((1,1)), size= round(len(df) * 0.5))
        cov = MinCovDet(random_state=0).fit(df) #calculate covariance
        mcd = cov.covariance_ #robust covariance metric
        robust_mean = cov.location_  #robust mean
        inv_covmat = sp.linalg.inv(mcd) #inverse of covariance matrix
    else:
        X = rng.multivariate_normal(mean=np.mean(df, axis=0), cov=real_cov, size= round(len(df) * 0.5))
        cov = MinCovDet(random_state=0).fit(X) #calculate covariance
        mcd = cov.covariance_ #robust covariance metric
        robust_mean = cov.location_  #robust mean
        inv_covmat = sp.linalg.inv(mcd) #inverse of covariance matrix

    # Robust M-Distance
    x_minus_mu = df - robust_mean
    print(x_minus_mu)
    # Transform data into dask arrays
    x_minus_mu_dask = da.from_array(x_minus_mu.to_numpy(), chunks=(min(x_minus_mu.shape[0], 10000), df.shape[1]))
    mahal = da.sqrt(da.diagonal(da.dot(da.dot(x_minus_mu_dask,inv_covmat), x_minus_mu_dask.T)))
    # Calculate md
    md = mahal.compute()
    # Compute the chi-squared cumulative probability distribution to transfer the md2 into probabilities
    probability_md = 1 - chi2.cdf(md, df=df.shape[1])

    # Save md values and probabilities to df column
    md_df = pd.DataFrame({md_name:md,p_md_name:probability_md})
    # Set a Chi2 cut-off point using probability of 0.01 (99.5% Chi2 quantile)
    # Degrees of freedom (df) = number of variables
    threshold = chi2.ppf((1-cut), df=df.shape[1]-1)
    print(threshold)
    # STD threshold, assuming 'md' contains your computed distances
    # mean_md = np.mean(md)
    # std_md = np.std(md)
    # threshold_3std = mean_md + 4 * std_md
    # Flag outliers as md > threshold
    md_df[outlier_name] = md_df[md_name] > threshold
    return md_df

In [25]:
def mahalanobis_per_condition(df, cut=0.001):
    # Select unique uids
    conditions = df['ExperimentalCondition'].unique()

    # Empty df to save all md and outliers
    md_df = pd.DataFrame()

    for i, condition in enumerate(conditions):
        # Select the uid data subset
        df_uid = df[df['ExperimentalCondition'].isin([condition])].reset_index(drop=True)
        print(df_uid)
        # ---- EYE columns ----
        # mahalanobis distances for each data subset
        mds_eye= robust_mahalanobis_method_dask(df=df[['EyeDirWorldCombined.x','EyeDirWorldCombined.y','EyeDirWorldCombined.z']], md_name='md_eye',outlier_name='eye_outlier',p_md_name='p_md_head', cut=cut).reset_index(drop=True)

        ## ---- HEAD columns ----
        mds_head= robust_mahalanobis_method_dask(df=df_uid[['NoseVector.x','NoseVector.y', 'NoseVector.z']], md_name='md_head_condition',outlier_name='head_outlier_condition', p_md_name='p_md_head_condition', cut=0.001).reset_index(drop=True)
        #
        # ## ---- CAR columns ----
        # mds_car= robust_mahalanobis_method_dask(df=df_uid[['CarYaw_degrees', 'CarPitch_degrees', 'CarRoll_degrees']], md_name='md_car_condition',outlier_name='car_outlier_condition', p_md_name='p_md_car_condition', cut=0.001).reset_index(drop=True)
        #
        # ## ---- Steering ----
        # mds_steer= robust_mahalanobis_method_dask(df=df_uid[['streeringDegree']], md_name='md_steering_condition',outlier_name='steering_outlier_condition', p_md_name='p_md_steer_condition', cut=0.001).reset_index(drop=True)
        #
        # ## ---- Pupil dilation ----
        # mds_pupil= robust_mahalanobis_method_dask(df=df_uid[['Pupil_Dilation_mean']], md_name='md_pupil_dilation_condition',outlier_name='pupil_dilation_outlier_condition', p_md_name='p_md_pupil_condition', cut=0.001).reset_index(drop=True)

        ## save md and outlier data
        # md_df = pd.concat([md_df, pd.concat([mds_eye,mds_head,mds_car,mds_steer,mds_pupil], axis=1)], ignore_index=True)
        md_df = pd.concat([md_df, pd.concat([mds_eye,mds_head], axis=1)], ignore_index=True)
    ## Concatenate original df with calculated mahalanobis distances and outliers
    # df = pd.concat([df.reset_index(drop=True), mds_eye.reset_index(drop=True)], axis=1)
    df = pd.concat([df.reset_index(drop=True), md_df.reset_index(drop=True)], axis=1)
    return df

In [26]:
import numpy as np
import pandas as pd
import dask.array as da
import scipy.linalg as sp_linalg
from sklearn.covariance import MinCovDet
from scipy.stats import chi2
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse


def robust_mahalanobis_method_dask2(df, md_name='md', outlier_name='outlier', p_md_name='p_md', cut=0.001):
    # Minimum covariance determinant
    rng = np.random.RandomState(0)
    real_cov = np.cov(df.values.T)

    # Calculate inverse covariance robustly
    # if df.shape[1] < 2:
    #     cov = MinCovDet(random_state=0).fit(df.values.reshape(-1,1))
    #     mcd = cov.covariance_
    #     robust_mean = cov.location_
    #     inv_covmat = sp_linalg.inv(mcd)
    # else:
    # X = rng.multivariate_normal(mean=np.mean(df, axis=0), cov=real_cov, size= round(len(df) * 0.5))
    cov = MinCovDet(random_state=0).fit(df.values)
    mcd = cov.covariance_
    robust_mean = cov.location_
    inv_covmat = sp_linalg.inv(mcd)

    # Calculate robust Mahalanobis distances
    x_minus_mu = df.values - robust_mean
    x_minus_mu_dask = da.from_array(x_minus_mu, chunks=(min(x_minus_mu.shape[0], 10000), df.shape[1]))
    mahal_sq = da.diagonal(x_minus_mu_dask @ inv_covmat @ x_minus_mu_dask.T)
    md = da.sqrt(mahal_sq).compute()

    # Calculate probability using Chi2 distribution
    p_md = 1 - chi2.cdf(mahal_sq.compute(), df=df.shape[1])

    # Threshold
    threshold = chi2.ppf(1 - cut, df=df.shape[1])

    # Compile result DataFrame
    md_df = pd.DataFrame({
        md_name: md,
        p_md_name: p_md
    })
    md_df[outlier_name] = mahal_sq.compute() > threshold

    return md_df

## Useful functions

In [27]:
# Mapping of old labels to new labels
def map_labels(labels, label_mapping):
    """
    Maps each label in the given list using the provided mapping dictionary.

    Args:
        labels (list): List of original labels.
        label_mapping (dict): Mapping dictionary for label replacement.

    Returns:
        list: New list with mapped labels.
    """
    return [label_mapping.get(label, label) for label in labels]

# Define your label mapping (you can modify this as needed)
custom_mapping = {
    'EyeDirWorldCombined.x': 'Eye Horizontal',
    'EyeDirWorldCombined.y': 'Eye Vertical',
    'NoseVector.x': 'Head Horizontal',
    'NoseVector.y': 'Head Vertical',
    'SteeringInput': 'Steering',
    'ExperimentalCondition ': 'Condition',
    'SuccessfulCompletionState': 'SuccessfulCompletion'
}

In [28]:
def biplot(score, coef, eigenvalues,
           labels=None,
           color_map=None,
           colors=None,
           explained_variance=None,
           vector_colors=None,
           scaled=None,
           vector_linewidth=None,
           legend_by='condition', # new parameter with default 'condition'
           **kwargs,):
    # plt.rcParams.update({'font.size': 10})
    # Apply 90-degree rotation matrix to `score` and `coef`
    rotation_matrix = np.array([[0, -1], [1, 0]])
    # Rotate the first two components of the scores
    # score = np.dot(score[:, :2], rotation_matrix)
    # Rotate the first two components of the loadings
    # coef = np.dot(coef[:, :2], rotation_matrix)
    xs = score[:, 0]
    ys = score[:, 1]
    n = coef.shape[0]
    scalex = 1.0 / (xs.max() - xs.min())
    scaley = 1.0 / (ys.max() - ys.min())

    padding= 1.2 # 20% padding for axis and vector scaling
    padding_text = 1 # 10 % padding for text
    xlims = padding * np.max(np.abs(xs))
    ylims = padding * np.max(np.abs(ys))
    if colors is None:  # If no color information, plot all points orange
        plt.scatter(xs, ys, s=80, color='gray', alpha=0.5,edgecolor='gray')
        # plt.scatter(xs, ys, s=80, color='orange')
    else:  # If color information is given, plot points with corresponding colors
        # plt.scatter(xs * scalex, ys * scaley, s=80, color=colors, alpha=0.5)
        plt.scatter(xs, ys, s=80, color=colors, alpha=0.5,edgecolor=colors)
    plt.xlim(-xlims * padding,xlims * padding)
    plt.ylim(-ylims* padding,ylims* padding)

    # Adjust the number of labels to match the number of coefficients
    original_labels = labels[:n]

    # Call the function to get the new mapped labels
    labels = map_labels(original_labels, custom_mapping)

    # Draw principal component vectors as arrows
    for i in range(n):
        # Calculate arrow end point (arrow head)
        arrow_end_x = xlims * coef[i, 0]
        arrow_end_y = ylims * coef[i, 1]
        if scaled:
            # Draw the arrow starting from the origin (0, 0) to the arrow head
            plt.arrow(0, 0, arrow_end_x, arrow_end_y, color=vector_colors[i], alpha=1, linewidth=vector_linewidth,length_includes_head=True,head_length=0.1,head_width=0.1,overhang=0)
        else:
            plt.arrow(0, 0, arrow_end_x, arrow_end_y, color=vector_colors[i], alpha=1, linewidth=vector_linewidth,length_includes_head=True,head_length=0.7,head_width=0.7,overhang=0)
    # Position the label above the arrow head
        if labels is not None:
            if arrow_end_y > 0:
                va='bottom'
            else:
                va='top'
            # Annotate the point with text (you can customize the text as needed)
            plt.text(arrow_end_x * padding_text, arrow_end_y * padding_text,
                     f"{labels[i]}\n({coef[i, 0]:.2f}, {coef[i, 1]:.2f})",color='black',ha='center', va=va,fontdict=dict(fontsize=14), bbox=dict(facecolor='white', alpha=0.0001, edgecolor='white'))
    # Write first two eigenvalues
    # plt.text(-ylims + padding_text, -xlims * padding_text, "Eigenvalues:\n {:.2f}, {:.2f}".format(*eigenvalues),ha='left', va='bottom', fontsize=10)
    # Write all eigenvalues
    # plt.text(-ylims + 0.5, -xlims + 0.5, "Eigenvalues:\n " + ", ".join(f"{val:.2f}" for val in eigenvalues), ha='left', va='bottom', fontsize=10)

    # Set fixed axis limits
    ax = plt.gca()
    fixed_x_min, fixed_x_max = -xlims, xlims  # Your specified x limits
    fixed_y_min, fixed_y_max = -ylims, ylims  # Your specified y limits

    # Set the fixed limits
    plt.xlim(fixed_x_min, fixed_x_max)
    plt.ylim(fixed_y_min, fixed_y_max)

    # Hide all spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)

    # Parameters for arrows
    arrow_length = 0.15  # Size of the arrowhead
    arrow_linewidth = 2.5

    # Create y-axis arrow (vertical) ON THE LEFT SPINE
    #  Start at bottom left corner
    ax.arrow(fixed_x_min +arrow_length, fixed_y_min+arrow_length,
             0, fixed_y_max - (fixed_y_max/2.5),  # Go up to max - arrowhead space
             head_width=arrow_length, head_length=arrow_length,
             fc='black', ec='black', linewidth=arrow_linewidth,
             length_includes_head=True)

    # Create x-axis arrow (horizontal) ON THE BOTTOM SPINE
    # Start at bottom left corner
    ax.arrow(fixed_x_min +arrow_length, fixed_y_min+arrow_length,
             fixed_x_max - (fixed_x_max/2.5), 0,  # Go right to max - arrowhead space
             head_width=arrow_length, head_length=arrow_length,
             fc='black', ec='black', linewidth=arrow_linewidth,
             length_includes_head=True)
    # Hide tick parameters
    ax.set_xticks([])
    ax.set_yticks([])

    # Remove standard labels
    ax.set_xlabel('')
    ax.set_ylabel('')

    # Add custom positioned labels
    # X-axis label - at the start of the x-axis arrow
    ax.text(fixed_x_min + 1.5, fixed_y_min - 0.1,  # Position just below the arrow end
            "PC{} ({:.1f}%)".format(1, explained_variance[0] * 100),
            fontsize=10, ha='right', va='top')

    # Y-axis label - at the start of the y-axis arrow
    ax.text(fixed_x_min + 0.02, fixed_y_min + 0.6,  # Position to the left of arrow end
            "PC{} ({:.1f}%)".format(2, explained_variance[1] * 100),
            fontsize=10, ha='right', va='bottom', rotation=90)

    # Add a legend for the colors if provided
    # Create a mapping from condition to color
    # Add a legend for the condition labels based on their colors
    # Automatic legend based on color_map
    # Generate legend based on specified criteria
    if colors is not None:
        import collections
        handles = []

        if legend_by == 'condition' and color_map is not None:
            # Build reverse map from color to condition
            color_to_condition = {v: k for k, v in color_map.items()}
            unique_colors = list({c for c in colors})
            for col in unique_colors:
                label = color_to_condition.get(col, col)
                handle = Line2D([0], [0], marker='o', color='w',
                                markerfacecolor=col, markersize=10,
                                label=label)
                handles.append(handle)
            plt.legend(handles=handles, title = legend_by.capitalize(), fontsize=10, title_fontsize= 12, loc='upper right')


        elif legend_by == 'completion':
            # Based on 'SuccessfulCompletionState' column
            # Create a mapping for completion status if not provided
            # Assign green for success (==1), red otherwise
            for cond_value in set(kwargs.get('completion_states', [])):
                if cond_value == 0:
                    col = 'red'
                    label = 'Failure'
                elif cond_value == 1:
                    col = 'green'
                    label = 'Success'
                handle = Line2D([0], [0], marker='o', color='w',
                                markerfacecolor=col, markersize=10,
                                label=label)
                handles.append(handle)
            plt.legend(handles=handles,title = legend_by.capitalize(), fontsize=10, title_fontsize= 12, loc='upper right')
            # sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1), fontsize=10)
    # plt.tight_layout()

In [29]:
def get_pca_var(pca, subset_scaled, feature_names):
    # Validate input types
    if not isinstance(pca, PCA):
        raise TypeError("Expected a PCA object from sklearn.decomposition.PCA")

    # Compute coordinates
    coords = pca.components_.T * np.sqrt(pca.explained_variance_)

    # Compute correlations
    # Since we're using standardized data:
    cor = coords / np.std(subset_scaled, axis=0)

    # Compute cos2 (squared loadings or cosine similarity)
    cos2 = np.square(cor)

    # Compute contributions
    total_variance = np.sum(pca.explained_variance_)
    contrib = (cos2 * 100 * pca.explained_variance_) / total_variance

    # Create dataframes with feature names as the index
    coord_df = pd.DataFrame(coords, index=feature_names, columns=[f'PC{i+1}' for i in range(pca.n_components_)])
    cor_df = pd.DataFrame(cor, index=feature_names, columns=[f'PC{i+1}' for i in range(pca.n_components_)])
    cos2_df = pd.DataFrame(cos2, index=feature_names, columns=[f'PC{i+1}' for i in range(pca.n_components_)])
    contrib_df = pd.DataFrame(contrib, index=feature_names, columns=[f'PC{i+1}' for i in range(pca.n_components_)])

    return {
        'coord': coord_df,
        'cor': cor_df,
        'cos2': cos2_df,
        'contrib': contrib_df
    }

In [30]:
from matplotlib.patches import Circle
from matplotlib import colors as mcolors

def plot_cosine_similarity(cos2_df, title='', time_point='', save=False):
    # Extract feature names directly from the DataFrame index
    feature_names = cos2_df.index

    # max_cos2_value = 1
    # print(f'Plotting cosine similarity...')
    max_cos2_value = cos2_df.values.max() # color bar maximum limit
    # Create a custom colormap from white to #A2530E
    cmap = mcolors.LinearSegmentedColormap.from_list("", ["white", "#A2530E"])

    features, components = cos2_df.shape
    fig, ax = plt.subplots(figsize=(8, 4))
    if len(feature_names) > 10:
        num_fontsize = 10
    num_fontsize = 12
    ax.set_title(title, fontsize=18)

    # Plot each circle and add annotations
    for i in range(features):
        for j in range(components):
            value = cos2_df.iloc[i, j]
            color = cmap(value / max_cos2_value)  # Normalize the color
            circle = Circle((j, i), radius=np.sqrt(value) * 0.5, color=color, fill=True)
            ax.add_artist(circle)

            # Annotate the cos2 value at the center of the circle
            ax.text(
                j, i, f'{value:.2f}',
                color='lightgray',
                ha='center', va='center',
                fontsize=num_fontsize,  # Adjust font size for readability
            )

    # Setup axis limits and labels
    ax.set_xlim(-0.5, components - 0.5)
    ax.set_ylim(-0.5, features - 0.5)
    ax.set_xticks(np.arange(components))
    ax.set_yticks(np.arange(features))
    ax.set_xticklabels([f'PC{i+1}' for i in range(components)], rotation=0)
    ax.set_yticklabels(feature_names)
    ax.set_aspect('equal', 'box')

    # Add grid lines for clarity
    ax.hlines(np.arange(-0.5, features), xmin=-0.5, xmax=components - 0.5, color='grey', lw=0.5)
    ax.vlines(np.arange(-0.5, components), ymin=-0.5, ymax=features - 0.5, color='grey', lw=0.5)

    # Set outer border to gray by modifying the spines
    for spine in ax.spines.values():
        spine.set_edgecolor('grey')
        spine.set_linewidth(0.5)

    # Add a color bar on the right
    norm = mcolors.Normalize(vmin=0, vmax=max_cos2_value)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, orientation="vertical", fraction=0.046, pad=0.04, label='$Cos^2$')

    # Increase the number of ticks on the color bar for better granularity
    num_ticks = 6
    cbar.set_ticks(np.linspace(0, max_cos2_value, num_ticks))
    cbar.set_ticklabels([round(val, 2) for val in np.linspace(0, max_cos2_value, num_ticks)])
    cbar.outline.set_visible(True)
    plt.tight_layout()
    # if save:
    #     plt.savefig(f"plots/cosine/pca_cos2_{event}_{time_point}_{''.join(map(str, feature_names))}.pdf", dpi=1200)
    # plt.show()

In [44]:
# Directories to save results
# Ensure os is imported to work with the file system
import os
from tqdm import tqdm  # Import the tqdm library for progress bar
import time  # Import time for simulating or handling elapsed time calculation

# Define the base results path
results_path = "data/pca_results"

# Create base results directory
os.makedirs(results_path, exist_ok=True)

# Create subdirectories under the base results path
directories = {
    "cleaned_new": os.path.join(results_path, "cleaned_new"),
    "variances": os.path.join(results_path, "variances"),
    "components": os.path.join(results_path, "components"),
    "eigenvalues": os.path.join(results_path, "eigenvalues"),
}

# Create each subdirectory
for subdir in directories.values():
    os.makedirs(subdir, exist_ok=True)

def visualize_event_pca(df, event='', time_point='', d_ts = 0, features='', scaled=True, vis_biplot=True,output_mode = 'show', legend_mode='condition',save_biplot=False, save_cos2_plot=False, save_cleaned=False, direc_path=''):
    if not event == '':
        # Find the actual event name in the dataframe that contains the provided event name
        available_events = df['EventName'].unique()
        matching_events = [e for e in available_events if event in e]
        
        if not matching_events:
            raise ValueError(f"No event found containing '{event}'. Available events: {available_events}")
        elif len(matching_events) > 1:
            print(f"Warning: Multiple events found containing '{event}': {matching_events}. Using the first one: {matching_events[0]}")
        
        actual_event_name = matching_events[0]
        print(f"Using event: {actual_event_name}")
        
        # Filter data for the actual event
        event_df = df[df['EventName'] == actual_event_name]
        
        if not time_point == '':
            if time_point == 'start':
                # event start time
                ts = event_df['time_from_event'].min()
            elif time_point == 'end':
                # event end time
                ts = event_df['time_from_event'].max()
            elif time_point == 'onset':
                # event end time
                ts = 0
            elif time_point == 'dynamic':
                ts = d_ts
            # timestamps = event_df['time'].unique()
            timestamps = event_df[np.isclose(event_df['time_from_event'], ts, atol=0.00001)]['time_from_event'].unique()
        else:
            # timestamp subset of the entire event
            timestamps = event_df['time_from_event'].unique()
    else:
        event = 'all_events'
        # timestamp subset of the entire df
        timestamps = df['time_from_event'].unique()

    # DataFrames to save data
    df_cleaned_ts = pd.DataFrame()
    df_event_components = pd.DataFrame()
    df_event_variance = pd.DataFrame()
    df_pca_results = pd.DataFrame()
    df_sorted_eigenvalues = pd.DataFrame()  # To store sorted eigenvalues

    with (tqdm(total=len(timestamps), desc="Processing times", unit="", colour="green") as pbar):
        start_time = time.time()  # Store the starting time
        for i, timestamp in enumerate(timestamps):
            # Clear the previous plot before drawing the new one
            # ax.clear()
            # 0. Extract rows for this timestamp
            subset = df[df['time_from_event'] == timestamp]
            event_name = subset['EventName'].unique()[0]

            # 1. Update progress bar for the current timestamp
            pbar.set_postfix_str(f"Current Timestamp: {timestamp}")

            # 2. Clean outliers in subset
            subset_cleaned = subset#.pipe(clean_subset)

            # 3. Features
            subset_features1 = subset_cleaned[subset_cleaned.columns.intersection(features)]
            subset_features_renamed = subset_features1.rename(columns=custom_mapping)
            subset_features = subset_features_renamed.drop(columns=['ExperimentalCondition','SuccessfulCompletion'])
            original_labels = list(subset_features.columns)

            completion_states = list(subset['SuccessfulCompletionState'].unique())

            # 3.1 Colors
            if legend_mode == 'completion':
                color_map = {
                    1.0 : 'green',
                    0.0 : 'red',
                }
                colors = [color_map.get(condition, 'black') for condition in subset_features_renamed['SuccessfulCompletion']]
            else:
                color_map = {
                        'BaseCondition': 'gray',
                        'FullLoopAR': '#cc2936',
                        'HUDOnly': '#1f77b4',   # example blue
                        'AudioOnly': '#ff7f0e'  # example orange
                    }

                colors = [color_map.get(condition, 'black') for condition in subset_features_renamed['ExperimentalCondition']]

            # 4. Scaling
            if scaled:
                scaler = StandardScaler(with_std=True)
                subset_scaled = scaler.fit_transform(subset_features)
            else:
                subset_scaled = subset_features

            # 5. Perform PCA
            pca = PCA(whiten=True)#, svd_solver='full')
            pca_result = pca.fit_transform(subset_scaled)
            explained_variance = pca.explained_variance_ratio_
            eigenvalues = pca.explained_variance_

            # 5.1 Sorted eigenvalues
            sorted_eigenvalue_columns = [f'eigen_val{i+1}' for i in range(len(eigenvalues))]
            eigenvalues_df = pd.DataFrame([eigenvalues], columns=sorted_eigenvalue_columns)
            eigenvalues_df['Time'] = timestamp
            eigenvalues_df['EventName'] = event_name
            df_sorted_eigenvalues = pd.concat([df_sorted_eigenvalues, eigenvalues_df], ignore_index=True)

            # Save PCA results
            pca_result_df = pd.DataFrame(
                pca_result,
                columns=[f'PC{i+1}' for i in range(len(explained_variance))]
            )
            pca_result_df['Time'] = timestamp
            pca_result_df['EventName'] = event_name
            df_pca_results = pd.concat([df_pca_results, pca_result_df], ignore_index=True)

            # Components
            loadings = pd.DataFrame(
                pca.components_.T,
                columns=[f'PC{i+1}' for i, v in enumerate(explained_variance)],
            )
            loadings['Features'] = original_labels
            loadings['Time'] = timestamp
            loadings['EventName'] = event_name
            df_event_components = pd.concat([df_event_components, loadings], ignore_index=True)

            # Variance
            df_variances = pd.DataFrame(
                np.reshape(explained_variance, (1, len(explained_variance))),
                columns=[f'PC{i+1}' for i, v in enumerate(explained_variance)]
            )
            df_variances['Time'] = timestamp
            df_variances['EventName'] = event_name
            df_event_variance = pd.concat([df_event_variance, df_variances], ignore_index=True)

            # Visualization
            if vis_biplot:
                sns.set_theme(style="white")
                plt.figure(figsize=(8, 6))
                plt.title('Second: ' + str(timestamp), fontsize=22)
                biplot(
                    pca_result,
                    pca.components_.T,
                    eigenvalues,
                    original_labels,
                    color_map,
                    colors,
                    explained_variance,
                    vector_colors=['k'] * len(original_labels),
                    scaled=scaled,
                    vector_linewidth=1.1,
                    legend_by=legend_mode,
                    completion_states=completion_states
                )
                # --- Display or save biplot ---
                if output_mode == 'save':
                    if save_biplot:
                        save_path = f"plots/biplot/pca_{event}_{timestamp}_{''.join(map(str, original_labels))}.png"
                        os.makedirs(os.path.dirname(save_path), exist_ok=True)
                        plt.savefig(save_path, dpi=1200)
                        # print(f"Biplot saved to {save_path}")
                    plt.close() # Always close in save mode
                elif output_mode == 'show':
                    plt.show() # Show the plot and wait for user to close it
                else:
                    plt.close() # Default action is to close to prevent memory leaks
                    # print("Biplot created, saved, and closed.")

            # --- Block for Cosine Similarity Plot ---
            # plt.figure(figsize=(8, 4)) # Creates Figure B, makes it active
            # plt.close()
            pca_cos2 = get_pca_var(pca, subset_scaled, original_labels)
            cos2_matrix = pca_cos2['cos2']
            plot_cosine_similarity(cos2_matrix, title='Cosine Similarity ($Cos^2$)') # Plots on Figure B

            # --- Display or save cosine plot ---
            if output_mode == 'save':
                if save_cos2_plot:
                    save_path = f"plots/cosine/pca_cos2_{event}_{timestamp}_{''.join(map(str, original_labels))}.png"
                    os.makedirs(os.path.dirname(save_path), exist_ok=True)
                    plt.savefig(save_path, dpi=1200)
                    # print(f"Cosine similarity plot saved to {save_path}")
                plt.close() # Always close in save mode
            elif output_mode == 'show':
                plt.show() # Show the plot
            else:
                plt.close() # Default action is to close
                # print("Cosine plot created and closed.")
            plt.close()
            # finally, capture this frame
            # camera.snap()

            # Cleaned subset
            if save_cleaned:
                subset_features_renamed['EventName'] = event_name
                subset_features_renamed['Time'] = timestamp
                df_cleaned_ts = pd.concat([df_cleaned_ts, subset_features_renamed], ignore_index=True)

            # Update progress bar and elapsed time
            elapsed_time = time.time() - start_time
            pbar.update(1)
            pbar.set_postfix_str(f"Elapsed: {elapsed_time:.2f}s")

    # Save results
    if save_cleaned:
        # Determine the filename suffix based on whether we're processing a specific event or all events
        if event != 'all_events':
            # For specific events, use the provided event name and 'all' timestamps
            timestamp_suffix = 'all'
            event_suffix = event  # Use the simplified event name provided by user
        else:
            # For all events, use 'all_events' and 'all' timestamps
            timestamp_suffix = 'all'
            event_suffix = 'all_events'
        
        df_cleaned_ts.to_csv(f"{direc_path['cleaned_new']}/cleaned_event_{event_suffix}_timestamp_{timestamp_suffix}_{len(original_labels)}_variables_.csv", index=False)
        df_event_variance.to_csv(f"{direc_path['variances']}/variances_event_{event_suffix}_timestamp_{timestamp_suffix}_{len(original_labels)}_variables.csv", index=False)
        df_event_components.to_csv(f"{direc_path['components']}/components_event_{event_suffix}_timestamp_{timestamp_suffix}_{len(original_labels)}_variables.csv", index=False)
        df_sorted_eigenvalues.to_csv(f"{direc_path['eigenvalues']}/eigenval_event_{event_suffix}_timestamp_{timestamp_suffix}_{len(original_labels)}_variables.csv", index=False)
    # After the loop – create and display the animation
    # animation = camera.animate(blit=False, interval=500)
    # display(HTML(animation.to_html5_video()))
    return df_event_components, df_event_variance, original_labels, df_cleaned_ts, df_pca_results, df_sorted_eigenvalues


In [32]:
df_single_event.columns

Index(['timestamp_REF', 'uid', 'dataset', 'city_section',
       'ExperimentalCondition', 'UnixTimeStamp', 'TobiiTimeStamp',
       'RightEyeIsBlinkingWorld', 'RightEyeIsBlinkingLocal',
       'LeftEyeIsBlinkingWorld', 'LeftEyeIsBlinkingLocal', 'HmdPosition.x',
       'HmdPosition.y', 'HmdPosition.z', 'NoseVector.x', 'NoseVector.y',
       'NoseVector.z', 'EyePosWorldCombined.x', 'EyePosWorldCombined.y',
       'EyePosWorldCombined.z', 'EyeDirWorldCombined.x',
       'EyeDirWorldCombined.y', 'EyeDirWorldCombined.z',
       'EyePosLocalCombined.x', 'EyePosLocalCombined.y',
       'EyePosLocalCombined.z', 'EyeDirLocalCombined.x',
       'EyeDirLocalCombined.y', 'EyeDirLocalCombined.z', 'ObjectName_1',
       'ObjectName_2', 'ObjectName_3', 'ObjectName_4', 'ObjectName_5',
       'HitObjectPosition.x_1', 'HitObjectPosition.x_2',
       'HitObjectPosition.x_3', 'HitObjectPosition.x_4',
       'HitObjectPosition.x_5', 'HitObjectPosition.y_1',
       'HitObjectPosition.y_2', 'HitObjectPositio

In [33]:
conditions = ['ExperimentalCondition','SuccessfulCompletionState']
eye = ['EyeDirWorldCombined.x', 'EyeDirWorldCombined.y']#, 'EyeDirLocalCombined.z']
head = ['NoseVector.x', 'NoseVector.y']#, 'NoseVector.z']
steering = ['SteeringInput']
variables_to_use = conditions + eye + head #+ steering

In [34]:
import logging
logging.getLogger('matplotlib.font_manager').disabled = True

import ipywidgets as widgets  # interactive display
%config InlineBackend.figure_format = 'svg'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/nma.mplstyle")

# Saving data
1. You can work with 1 **event** by using the name parameter `event = 'StagEventNew'`, or all the entire data by `event = ''`

2. In case you want to **save data**, then change parameter `output_mode = 'save'` in the function and
--> `save_biplot=True`, `save_cos2_plot=True`, and `save_cleaned=True`, `vis_biplot=False`
3. If you want to **visualize by* either successful completion state `legend_mode= 'completion'`, otherwise use `legend_mode= 'condition'`
4. Look for the results data in the respective folders and load it for the next steps

In [45]:
import ipywidgets as widgets  # Add import if not already present

event = 'StagEventNew'  # Process all data in the dataframe
time_point = 'dynamic'
scaled = True
timestamps = sorted(df_single_event['time_from_event'].unique())

# Create global variables to store outputs
components_df = None
variances_df = None
original_labels = None
df_cleaned_ts = None
df_pca_results = None
eigenvalues_df = None

# First, save the complete event data (this happens once, independent of the widget)
print("Saving complete event data...")
components_df, variances_df, original_labels, df_cleaned_ts, df_pca_results, eigenvalues_df = visualize_event_pca(
    df_single_event,
    event=event,
    time_point='',  # Process all timestamps for saving
    d_ts=0,  # Not used when time_point is empty
    features=variables_to_use,
    scaled=scaled,
    vis_biplot=False,  # No visualization for saving
    output_mode='save',
    legend_mode='completion',
    save_biplot=False,  # No individual biplots for saving
    save_cos2_plot=False,  # No individual cos2 plots for saving
    save_cleaned=True,  # Save the complete dataset
    direc_path=directories
)
print("Complete event data saved successfully!")

# Now create the interactive widget for visualization only
def interactive_pca(d_ts):
    # This function only handles visualization, no saving
    visualize_event_pca(
        df_single_event,
        event=event,
        time_point=time_point,
        d_ts=d_ts, # dynamic timestamp for slider
        features=variables_to_use,
        scaled=scaled,
        vis_biplot=True,
        output_mode='show',
        legend_mode='completion',
        save_biplot=False,  # No saving in widget
        save_cos2_plot=False,  # No saving in widget
        save_cleaned=False,  # No saving in widget
        direc_path=directories
    )

widgets.interact(
    interactive_pca,
    d_ts=widgets.FloatSlider(
        value=0.2, min=-5, max=5, step=0.02,
        description='Time',
        layout=widgets.Layout(width='800px')
    )
)

Saving complete event data...
Using event: StagEventNew


Processing times: 100%|██████████| 501/501 [00:17<00:00, 29.18/s, Elapsed: 17.17s]         


Complete event data saved successfully!


interactive(children=(FloatSlider(value=0.2, description='Time', layout=Layout(width='800px'), max=5.0, min=-5…

<function __main__.interactive_pca(d_ts)>

# Visualize Variance

1. Currently, the variables `components_df,components_df, variances_df, original_labels, df_cleaned_ts, df_pca_results, eigenvalues_df` contain all the results information for the selected event. If empty `event = '' `, then for all data.

2. Since these variables are being held in memory, you can use them to visualize the variances and effective dimensionality. To avoind having to run the whole script multiple times (when restarting the kernel, etc.) then, you can also load the data from the results folder.

In [47]:
components_df

,PC1,PC2,PC3,PC4,Features,Time,EventName
0,-0.697631,0.166558,-0.169519,-0.675894,Head Horizontal,-5.00,StagEventNew
1,0.710770,0.040089,-0.189767,-0.676156,Head Vertical,-5.00,StagEventNew
2,0.054723,0.711631,-0.642056,0.279913,Eye Horizontal,-5.00,StagEventNew
3,-0.071563,-0.681347,-0.723199,0.087346,Eye Vertical,-5.00,StagEventNew
4,-0.711701,0.098029,-0.161178,-0.676679,Head Horizontal,-4.98,StagEventNew
...,...,...,...,...,...,...,...
1999,-0.471066,-0.496979,-0.694841,-0.219782,Eye Vertical,4.98,StagEventNew
2000,-0.612675,0.318085,-0.037831,0.722510,Head Horizontal,5.00,StagEventNew
2001,-0.387625,-0.599800,0.699429,-0.028015,Head Vertical,5.00,StagEventNew
2002,-0.504285,0.538683,0.156176,-0.656602,Eye Horizontal,5.00,StagEventNew


In [62]:
import pandas as pd

# Load the datasets from 'data/pca_results/'
import glob
import os

import os

base_dir = 'data/pca_results'

# List all folders inside base_dir
for folder_name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder_name)
    if os.path.isdir(folder_path):
        print(f"Folder: {folder_name}")
        files = os.listdir(folder_path)
        csv_files = [file_name for file_name in files if file_name.endswith('.csv')]
        if csv_files:
            for file_name in csv_files:
                print(f"  - {file_name}")
        else:
            print("  (No .csv files found)")

Folder: components
  - components_event_StagEventNew_timestamp_all_4_variables.csv
Folder: cleaned_new
  - cleaned_event_StagEventNew_timestamp_all_4_variables_.csv
Folder: eigenvalues
  - eigenval_event_StagEventNew_timestamp_all_4_variables.csv
Folder: variances
  - variances_event_StagEventNew_timestamp_all_4_variables.csv


In [64]:
folder = 'components'
folder_path = os.path.join(base_dir, folder)
csv_files = glob.glob(os.path.join(folder_path, 'components_event_StagEventNew_timestamp_all_4_variables.csv'))
print(f"Found {len(csv_files)} CSV files in /data/pca_results/components/:")
for f in csv_files:
    print(f"File: {os.path.basename(f)}")

if not csv_files:
    raise FileNotFoundError("No CSV files found in /data/pca_results/components/")

# Only allow a single file to be loaded; if multiple, warn and ask user to specify which
if len(csv_files) == 0:
    raise FileNotFoundError("No CSV files found in /data/pca_results/components/")
elif len(csv_files) == 1:
    components_df = pd.read_csv(csv_files[0])
else:
    print("Warning: Multiple CSV files found in /data/pca_results/components/:")
    for idx, f in enumerate(csv_files):
        print(f"{idx}: {os.path.basename(f)}")
    raise RuntimeError("Multiple component files found. Please specify which file to load by setting the correct path or removing extra files.")
components_df

Found 1 CSV files in /data/pca_results/components/:
File: components_event_StagEventNew_timestamp_all_4_variables.csv


,PC1,PC2,PC3,PC4,Features,Time,EventName
0,-0.697631,0.166558,-0.169519,-0.675894,Head Horizontal,-5.00,StagEventNew
1,0.710770,0.040089,-0.189767,-0.676156,Head Vertical,-5.00,StagEventNew
2,0.054723,0.711631,-0.642056,0.279913,Eye Horizontal,-5.00,StagEventNew
3,-0.071563,-0.681347,-0.723199,0.087346,Eye Vertical,-5.00,StagEventNew
4,-0.711701,0.098029,-0.161178,-0.676679,Head Horizontal,-4.98,StagEventNew
...,...,...,...,...,...,...,...
1999,-0.471066,-0.496979,-0.694841,-0.219782,Eye Vertical,4.98,StagEventNew
2000,-0.612675,0.318085,-0.037831,0.722510,Head Horizontal,5.00,StagEventNew
2001,-0.387625,-0.599800,0.699429,-0.028015,Head Vertical,5.00,StagEventNew
2002,-0.504285,0.538683,0.156176,-0.656602,Eye Horizontal,5.00,StagEventNew


In [ ]:
import matplotlib.pyplot as plt

# Example color palette you provided
color_pca = {
    'PC1': '#F72B2B', 'PC2': '#B10606',
    'PC3': '#962222', 'PC4': '#851E1E',
    'PC5': '#641717', 'PC6': '#430F0F',
    'PC7': '#3A0E0E', 'PC8': '#320C0C',
    'PC9': '#2B0A0A', 'PC10': '#240808',
    'PC11': '#1D0707', 'PC12': '#160505',
    'PC13': '#100303', 'PC14': '#0A0202',
    # 'PC15': '#050101'
}

# Select PCs to plot
pcs = ['PC1', 'PC2']#, 'PC3']#, 'PC4']#, 'PC5']

fig, ax= plt.subplots(figsize=(8, 6))

for pc in pcs:
    ax.plot(variances_df['Time'], variances_df[pc], label=pc, color=color_pca.get(pc, '#000000'))
# Remove the default legend as we'll create a custom one later
ax.legend()#.remove()
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1), fontsize=12)
sns.despine(ax=ax, offset=0, trim=False)
plt.xlabel('Time')
plt.ylabel('Variance')
plt.title(f'Variance of Principal Components for {components_df.Features.unique()} over Time')
# plt.legend(title='Principal Components')
plt.grid(True)
plt.tight_layout()
plt.show()

# Effective dimensionality

In [ ]:
def compute_ED_estimators(eigenvalues):
    """
    Compute Effective Dimensionality estimators from eigenvalues.
    Parameters:
    eigenvalues (list or array): List of eigenvalues.
    Returns:
    dict: Dictionary containing various ED estimators (n1, n2, nInf, nC).
    """
    output = {}

    # Make eigenvalues a numpy array for easier manipulation
    eigen_val = np.array(eigenvalues)

    # Compute ED estimators
    K = len(eigen_val)
    eigen_sum = np.sum(eigen_val)
    norm_eigen_val = eigen_val / eigen_sum

    # Calculate variance using the K/(K-1) adjustment
    eigen_var = np.var(eigen_val, ddof=0) * ((K - 1) / K)

    # Calculate the four different estimators
    output["n1"] = np.prod(norm_eigen_val ** (-norm_eigen_val))
    output["n2"] = (eigen_sum ** 2) / np.sum(eigen_val ** 2)
    output["nInf"] = eigen_sum / np.max(eigen_val)
    output["nC"] = K - ((K ** 2) / (eigen_sum ** 2)) * eigen_var

    return output

# Process eigenvalues_df DataFrame
def calculate_ED(df):
    """
    Process eigenvalues from a DataFrame to compute ED estimators per timestamp.
    Parameters:
    df (DataFrame): A DataFrame containing eigenvalues and a Time column.
    Returns:
    DataFrame: A new DataFrame containing ED estimators and the Time column.
    """
    results = []

    for _, row in df.iterrows():
        # Exclude Time, Event col and handle NaNs (if any)
        eigenvalues = row[:-2].dropna()
        time = row['Time']
        event = row['EventName']
        ed_estimators = compute_ED_estimators(eigenvalues)
        ed_estimators['Time'] = time
        ed_estimators['EventName'] = event
        results.append(ed_estimators)

    # Convert results to a DataFrame
    processed_df = pd.DataFrame(results)
    return processed_df

# Apply function to eigenvalues_df
ed_df = calculate_ED(eigenvalues_df)

# Display the resulting DataFrame
ed_df

In [ ]:
def visualize_effective_dimensionality(ed_df, title="Effective Dimensionality Over Time", features=None, save=False):
    """
    Visualizes effective dimensionality measures through time.

    Parameters:
    - ed_df: DataFrame containing ED estimates with columns ['Time', 'n1', 'n2', 'nInf', 'nC'].
    - title: Title of the plot (default: "Effective Dimensionality Over Time").
    - features: List of features used in the analysis (for title display).
    - save: Whether to save the plot as a file (default: False).

    Returns:
    - None
    """
    # Set style
    sns.set_theme(style="ticks")
    # Create the plot
    fig, ax = plt.subplots(figsize=(14, 8))

    # Create a mapping for LaTeX-style labels
    label_mapping = {
        'n1': '$n_{1}$',
        'n2': '$n_{2}$',
        'nInf': '$n_{\\infty}$',
        'nC': '$n_{C}$'
    }
    eddf = ed_df.drop(columns=['EventName'], inplace=False)

    # Convert ed_df to long format for lineplot
    ed_long = eddf.melt(id_vars=['Time'], var_name='Estimator', value_name='Value')

    # Map the estimators to their LaTeX-style labels
    ed_long['Estimator'] = ed_long['Estimator'].map(label_mapping)

    # Plot all lines at once, differentiating by hue and style
    # sns.lineplot(data=ed_long, x='Time', y='Value', hue='Estimator', style='Estimator',
    #              palette=sns.color_palette("gray", 4), linewidth=1.5, ax=ax_top)
    sns.lineplot(data=ed_long, x='Time', y='Value', hue='Estimator', style='Estimator',
                 palette=['#2F4858','#D83456', '#586A73', '#86888A'], linewidth=1.5, ax=ax)

    # Apply styling to top plot
    n_features = len(features) -2 # if features is not None else "unknown number of"
    ax.set_title(f"Effective dimensionality over time, {n_features} variables", fontsize=12)
    ax.set_ylabel("Effective Dimensionality", fontsize=10)

    # Hide the x-label on the top plot since we have a shared x-axis
    ax.set_xlabel("")

    # Remove the default legend as we'll create a custom one later
    # ax.get_legend()#.remove()
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1), fontsize=12)
    sns.despine(ax=ax, offset=0, trim=False)

    # Adjust layout to make room for the legend
    plt.tight_layout()

    # Show the plot
    plt.show()


In [ ]:
visualize_effective_dimensionality(ed_df, features=eigenvalues_df.columns, save=False)


# PCA Videos

In [143]:
import cv2

# Define the base results path
pca_path = "plots/"

# Create base results directory
os.makedirs(pca_path, exist_ok=True)

# Create subdirectories under the base results path
directories = {
    "videos": os.path.join(pca_path, "videos"),
    # "variances": os.path.join(results_path, "variances"),
    # "components": os.path.join(results_path, "components"),
    # "eigenvalues": os.path.join(results_path, "eigenvalues"),
}

# Create each subdirectory
for subdir in directories.values():
    os.makedirs(subdir, exist_ok=True)

images_folder = r'plots/biplot/'
video_name = "PCA_biplot_eye_head_StagEvent_1_7fps.mp4"
fps = 7.0


In [138]:
directories['videos']

'plots/videos'

In [146]:
# Use the biplots images to create a video
def make_video(image_folder, video_name, fps):
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    print(images)
    images.sort(key=lambda x: int(x.split('_')[5].split('.')[0]))

    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape


    # Use the 'mp4v' codec
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps,
                            (width, height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

In [147]:
make_video(images_folder, video_name, fps)

IndexError: list index out of range